<a href="https://colab.research.google.com/github/PigStep/CIFAR-10-based-Content-categorizator/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is this notebook about
This notebook tracks experiments, code and data manipulation for model

# Data loading

In [1]:
import torchvision
import torch
from torchvision.transforms import transforms

In [2]:
batch_size = 256

transform = transforms.ToTensor()

# Getting CIFAR-10 dataset with batches
trainset = torchvision.datasets.CIFAR10(root="/content",train=True,transform=transform,download=True)
trainloader = torch.utils.data.DataLoader(dataset = trainset,batch_size=batch_size,shuffle=True)

100%|██████████| 170M/170M [00:10<00:00, 16.0MB/s]


In [3]:
image_tensor, label = trainset[0]

print(f"Shape of tensor: {image_tensor.shape}")
print(f"Label: {label}")

Shape of tensor: torch.Size([3, 32, 32])
Label: 6


## Data normalization

This notebook uses torch's `ToTensor() + Normalize()` functions. Firstly we need to get some statistics before Normalization

In [4]:
channels_sum, channels_squared_sum, num_pixels = 0,0,0

for images, _ in trainloader:
  # images: [batch_size, channels, height, width]

  channels_sum += torch.sum(images, dim=[0,2,3])
  channels_squared_sum += torch.sum(images**2, dim = [0,2,3])
  num_pixels += images.numel()

mean = channels_sum / num_pixels
# std = sqrt(E[x^2] - (E[x])^2)
std = torch.sqrt((channels_squared_sum / num_pixels) - mean**2)

print(f"Channels means: {mean}")
print(f"Channels std: {std}")

Channels means: tensor([0.1638, 0.1607, 0.1488])
Channels std: tensor([0.2720, 0.2673, 0.2591])


In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,std=std)
])

# Getting datasets with normalization
trainset = torchvision.datasets.CIFAR10(root="/content",train=True,transform=transform,download=True)

testset = torchvision.datasets.CIFAR10(root="/content",train=False,transform=transform,download=True)
testloader = torch.utils.data.DataLoader(dataset = testset,batch_size=batch_size,shuffle=True)

# Baseline model: self constructed pytorch CNN

In [30]:
import torch.nn as nn
import torch.nn.functional as F

class baseCNN(nn.Module):
  def __init__(self):
    super().__init__()

    # 32x32 3 channels
    self.conv1 = nn.Conv2d(3, 32, 3)
    # 30x30 32 channels
    self.pool1 = nn.MaxPool2d(2,2)
    # 15x15 32 channels
    self.conv2 = nn.Conv2d(32, 64, 3)
    # 13x13 64 channels
    self.pool2 = nn.MaxPool2d(2,2)
    #6х6 64 channels
    self.fc = nn.Linear(2304,10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool1(x)

    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool2(x)

    x = torch.flatten(x,1)
    x = self.fc(x)

    return x


In [43]:
def train(baseCNN, train_loader, lr = 0.001, epochs = 100, thread_err = 0.1):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(baseCNN.parameters(), lr=lr)
    loss_list = []

    for epoch in range(epochs):
      total_loss = 0
      for image, label in train_loader:
        optimizer.zero_grad()

        prediction = baseCNN.forward(image)
        error = loss_fn(prediction, label)

        total_loss += error.item()

        error.backward()

        optimizer.step()

      avg_loss = total_loss / len(train_loader)
      loss_list.append(avg_loss)

      if(epoch % 10 == 0):
            print(f"Epoch: {epoch}, Error: {avg_loss}")

      if(avg_loss < thread_err):
          return loss_list

    return loss_list

In [44]:
CNN = baseCNN()
losses = train(CNN, trainloader)

KeyboardInterrupt: 